## 1. Import libraries

In [ ]:
import numpy
import pandas
from sklearn.datasets import make_regression

from scipy.stats import qmc
from scipy.special import chndtr, gammaincinv

from matplotlib import pyplot
from matplotlib.ticker import FixedLocator, FixedFormatter

import import_ipynb
import data_analysis

import matplotlib.pylab
params = {'legend.fontsize': 8,
         'font.family':'sans-serif',
         'font.sans-serif':'Times New Roman',
         'font.style':'normal',
         'font.weight':'normal',
         'axes.labelsize': 'xx-large',
         'axes.titlesize':'medium',
         'axes.spines.top': False,
         'axes.spines.right': False,
         'axes.linewidth': 1.0,
         'xtick.labelsize':'large',
         'ytick.labelsize':'large',
         'mathtext.fontset':'cm',
         'text.latex.preamble': r'usepackage{amsmath}',}

matplotlib.pylab.rcParams.update(params)
saved_img_path = './saved_images'
saved_img_path_clf_gn_given = './saved_images/classification/given'
saved_img_path_clf_gn_search = './saved_images/classification/search'
saved_img_path_reg_gn_given = './saved_images/regression/given'
saved_img_path_reg_gn_search = './saved_images/regression/search'

## 2. Influence analysis

In [ ]:
def influence_of_number_of_points_of_interest(list_of_n_points=None, perform_train=None, perform_val=None, perform_test=None, situation=None, perform='Accuracy', flag='clf', learning_type_flag='Chosen', lang='fr'):
    assert lang == 'fr' or lang == 'en'
    x_min, x_max = min(list_of_n_points), max(list_of_n_points)
    
    if flag == 'clf' :
        assert perform == 'Accuracy' or perform == 'Bound'
        if perform == 'Accuracy':
            y_min, y_max = 50.0, 100.0
            ylabel = 'Précision (%)' if lang == 'fr' else 'Accuracy (%)'
        else :
            y_min, y_max = 0, 0.5
            ylabel = 'Borne' if lang == 'fr' else 'Risk bound'
    else :
        assert perform == 'R2_scores' or perform == 'Bound'
        if perform == 'R2_scores' :
            y_min, y_max = 0.0, 1.0
            ylabel = 'Coefficient de détermination R2' if lang == 'fr' else 'R2 score'
        else :
            y_min, y_max = 0, 0.5
            ylabel = 'Borne' if lang == 'fr' else 'Risk bound'

    # figure
    pyplot.figure(figsize=(3,3), dpi=200)
    pyplot.plot(list_of_n_points, perform_train, '-*', lw=1, label='Training set')
    pyplot.plot(list_of_n_points, perform_val, '-*', lw=1, label='Validation set')
    pyplot.plot(list_of_n_points, perform_test, '-*', lw=1, label='Testing set')
    ax = pyplot.gca()
    ax.tick_params(direction="in")
    x_locator = FixedLocator([int(value) if numpy.floor(value)==numpy.ceil(value) else numpy.round(value,2) for value in numpy.arange(x_min,x_max+1)])
    y_locator = FixedLocator([int(value) if numpy.floor(value)==numpy.ceil(value) else numpy.round(value,2) for value in numpy.linspace(y_min,y_max,len(list_of_n_points))])
    x_formatter = FixedFormatter([str(int(value)) if numpy.floor(value)==value else str(numpy.round(value,2)) for value in numpy.arange(x_min,x_max+1)])
    y_formatter = FixedFormatter([str(int(value)) if numpy.floor(value)==value else str(numpy.round(value,2)) for value in numpy.linspace(y_min,y_max,len(list_of_n_points))])
    ax.xaxis.set_major_locator(x_locator)
    ax.yaxis.set_major_locator(y_locator)
    ax.xaxis.set_major_formatter(x_formatter)
    ax.yaxis.set_major_formatter(y_formatter)

    ax.set_xlabel('',fontsize='medium')
    ax.set_ylabel('',fontsize='medium')
    pyplot.xticks(fontsize=9)
    pyplot.yticks(fontsize=9)

    pyplot.xlabel(r'Number of points $n$')
    pyplot.ylabel(ylabel)
    pyplot.xlim(x_min,x_max+0.01)
    pyplot.ylim(y_min,y_max+0.01)
    pyplot.grid(linestyle = 'dashed', linewidth = 1)
    pyplot.legend(loc='lower left', fontsize=7, facecolor='white', edgecolor='black', framealpha=1.0)
    
    if learning_type_flag == 'Chosen':
        if flag == 'clf':
            pyplot.savefig(saved_img_path_clf_gn_given+'/img_influence_of_n_'+situation+'_'+perform+'_clf_gn_given.png', dpi=1000, bbox_inches='tight', transparent=True)
        else:
            pyplot.savefig(saved_img_path_reg_gn_given+'/img_influence_of_n_'+situation+'_'+perform+'_reg_gn_given.png', dpi=1000, bbox_inches='tight', transparent=True)
    else:
        if flag == 'clf':
            pyplot.savefig(saved_img_path_clf_gn_search+'/img_influence_of_n_'+situation+'_'+perform+'_clf_gn_search.png', dpi=1000, bbox_inches='tight', transparent=True)
        else:
            pyplot.savefig(saved_img_path_reg_gn_search+'/img_influence_of_n_'+situation+'_'+perform+'_reg_gn_search.png', dpi=1000, bbox_inches='tight', transparent=True)

In [ ]:
def Influence_of_Quasi_Monte_Carlo(n_features=2, dists=10, varepsilon=0.5, k=2, tau=0.1, low_bound=0, high_bound=1, index=0):
    dim, xmin, xmax, step = 1, 10, 100, 10
        
    Upsilon_seed = []
    for i in numpy.arange(500):
        sampler = qmc.Halton(dim, scramble=True, seed=i)
        Upsilon = []
        for n_row in numpy.arange(xmin, xmax+1, step):
            x_halton = sampler.random(n_row)
            BETA = gammaincinv(k, x_halton) / tau
            Upsilon.append(numpy.mean(chndtr((BETA / varepsilon)**2, n_features, (dists / varepsilon)**2)))
    
        Upsilon_seed.append(Upsilon)
    Upsilon_seed_mean, Upsilon_seed_std = numpy.mean(Upsilon_seed, axis=0), numpy.std(Upsilon_seed, axis=0)
    
    pyplot.figure(figsize=(2,2), dpi=200)
    pyplot.errorbar(numpy.arange(xmin, xmax+1, step), Upsilon_seed_mean, yerr=Upsilon_seed_std, marker='o', markersize=3.5, linestyle='dotted', capsize=3)
    # get min and max
    ymin, ymax = low_bound, high_bound
    
    ax = pyplot.gca()
    ax.tick_params(direction="in")
    x_locator = FixedLocator([int(value) if numpy.floor(value)==value else numpy.round(value,1) for value in numpy.linspace(xmin,xmax,10)])
    y_locator = FixedLocator([int(value) if numpy.floor(value)==value else numpy.round(value,3) for value in numpy.linspace(ymin,ymax,10)])
    x_formatter = FixedFormatter([str(int(value)) if numpy.floor(value)==value else str(numpy.round(value,1)) for value in numpy.linspace(xmin,xmax,10)])
    y_formatter = FixedFormatter([str(int(value)) if numpy.floor(value)==value else str(numpy.round(value,3)) for value in numpy.linspace(ymin,ymax,10)])
    ax.xaxis.set_major_locator(x_locator)
    ax.yaxis.set_major_locator(y_locator)
    ax.xaxis.set_major_formatter(x_formatter)
    ax.yaxis.set_major_formatter(y_formatter)
    
    ax.set_xlabel('',fontsize='small')
    ax.set_ylabel('',fontsize='small')
    pyplot.xticks(fontsize=8)
    pyplot.yticks(fontsize=8)

    pyplot.title(r'$(d, \left \| \mathbf{c}_{i_{0}} - \mathbf{x} \right \|, \varepsilon_{i}, k_{i}, \tau_{i}) = ($'+ str(n_features) + ', ' + str(dists) + ', ' + str(varepsilon) + ', ' + str(k) + ', ' + str(tau) + '$)$', fontsize=6)
    pyplot.xlabel(r'Sample size of $\beta_{i}$')
    pyplot.ylabel(r'$\Upsilon_{d, \left \| \mathbf{c}_{i_{0}} - \mathbf{x} \right \|, \varepsilon_{i}, k_{i}, \tau_{i}}$')
    pyplot.xlim(xmin-1,xmax+1)
    pyplot.ylim(ymin-1e-5,ymax+1e-5)
    pyplot.grid(linestyle = 'dashed', linewidth = 1)
    pyplot.savefig(saved_img_path+'/img_influence_of_quasi_monte_carlo_'+str(index)+'.png', dpi=1000, bbox_inches='tight', transparent=True)
    

## 3. Binary Linear Classification

#### Generate synthetic data

In [ ]:
def UniformRandomPointInRectangle(lengthRectangle=2, widthRectangle=1):
    return lengthRectangle * (2 * numpy.random.random() - 1), widthRectangle * (2 * numpy.random.random() - 1)

def ReplicateNTimes(func, lengthRectangle, widthRectangle, Ntrials=1000):
    numpy.random.seed(seed=0)
    xpoints, ypoints = [], []
    for _ in range(Ntrials):
        xp, yp = func(lengthRectangle, widthRectangle)
        xpoints.append(xp)
        ypoints.append(yp)
        
    Outputs = (xpoints, ypoints)
    return Outputs

In [ ]:
def ConstructTargetFromRectangleGN1(xpoints, ypoints, Xcenter, BETA, lengthRectangle, widthRectangle):
    assert BETA.size == 2
    
    xpoints, ypoints = numpy.array(xpoints), numpy.array(ypoints)
    class_ = numpy.zeros(xpoints.size)

    Conditions, RAD = [], []
    for i in range(BETA.size):
        rad = numpy.sqrt((xpoints - Xcenter[i,0])**2 + (ypoints - Xcenter[i,1])**2)
        RAD.append(rad)
        Conditions.append(rad <= BETA[i])

    RAD = numpy.array(RAD)
    Conditions.append(~((RAD[0] <= BETA[0]) | (RAD[1] <= BETA[1])))
    
    # REGION 1
    xpoints_1, ypoints_1 = xpoints[Conditions[0]], ypoints[Conditions[0]]
    class_1_ = class_[Conditions[0]]

    condition_neg_out_1 = (ypoints_1 > Xcenter[0,1])
    condition_pos_out_1 = (ypoints_1 < Xcenter[0,1])
    class_1_[condition_neg_out_1], class_1_[condition_pos_out_1] = -1, 1 

    if xpoints_1[condition_neg_out_1].size != xpoints_1[condition_pos_out_1].size:
        gap = xpoints_1[condition_neg_out_1].size - xpoints_1[condition_pos_out_1].size
        idx = numpy.where(condition_neg_out_1 == True)[0] if gap > 0 else numpy.where(condition_pos_out_1 == True)[0]
        choices = numpy.random.choice(idx, size=numpy.abs(gap), replace=False)
        xpoints_1 = numpy.delete(xpoints_1, choices)
        ypoints_1 = numpy.delete(ypoints_1, choices)
        class_1_ = numpy.delete(class_1_, choices)

    # REGION 2
    xpoints_2, ypoints_2 = xpoints[Conditions[1]], ypoints[Conditions[1]]
    class_2_ = class_[Conditions[1]]

    condition_neg_out_2 = (ypoints_2 > Xcenter[1,1])
    condition_pos_out_2 = (ypoints_2 < Xcenter[1,1])
    class_2_[condition_neg_out_2], class_2_[condition_pos_out_2] = -1, 1

    if xpoints_2[condition_neg_out_2].size != xpoints_2[condition_pos_out_2].size:
        gap = xpoints_2[condition_neg_out_2].size - xpoints_2[condition_pos_out_2].size
        idx = numpy.where(condition_neg_out_2 == True)[0] if gap > 0 else numpy.where(condition_pos_out_2 == True)[0]
        choices = numpy.random.choice(idx, size=numpy.abs(gap), replace=False)
        xpoints_2 = numpy.delete(xpoints_2, choices)
        ypoints_2 = numpy.delete(ypoints_2, choices)
        class_2_ = numpy.delete(class_2_, choices)

    # REGION EXT
    xpoints_out, ypoints_out = xpoints[Conditions[2]], ypoints[Conditions[2]]
    class_out_ = class_[Conditions[2]]

    condition_neg_out = (ypoints_out <= 0) 
    condition_pos_out = (ypoints_out > 0)
    class_out_[condition_neg_out], class_out_[condition_pos_out] = -1, 1

    if xpoints_out[condition_neg_out].size != xpoints_out[condition_pos_out].size:
        gap = xpoints_out[condition_neg_out].size - xpoints_out[condition_pos_out].size
        idx = numpy.where(condition_neg_out == True)[0] if gap > 0 else numpy.where(condition_pos_out == True)[0]
        choices = numpy.random.choice(idx, size=numpy.abs(gap), replace=False)
        xpoints_out = numpy.delete(xpoints_out, choices)
        ypoints_out = numpy.delete(ypoints_out, choices)
        class_out_ = numpy.delete(class_out_, choices)
    
    # for having same size
    condition_neg_out = (ypoints_out <= 0)
    condition_neg_out_1 = (ypoints_1 >= Xcenter[0,1])

    if xpoints_out[condition_neg_out].size != 2 * xpoints_1[condition_neg_out_1].size :
        gap = xpoints_out[condition_neg_out].size - 2 * xpoints_1[condition_neg_out_1].size
        if gap > 0 :
            # for neg out
            idx = numpy.where(condition_neg_out == True)[0]
            choices_neg_out = numpy.random.choice(idx, size=numpy.abs(gap), replace=False)
            # for pos out
            condition_pos_out = (ypoints_out > 0)
            idx = numpy.where(condition_pos_out == True)[0]
            choices_pos_out = numpy.random.choice(idx, size=numpy.abs(gap), replace=False)
            
            choices = numpy.concatenate((choices_neg_out, choices_pos_out), axis=0)
            xpoints_out = numpy.delete(xpoints_out, choices)
            ypoints_out = numpy.delete(ypoints_out, choices)
            class_out_ = numpy.delete(class_out_, choices)
        else:
            # for neg in
            idx = numpy.where(condition_neg_out_1 == True)[0]
            choices_neg_in = numpy.random.choice(idx, size=numpy.abs(int(gap/2)), replace=False)
            # for pos in
            condition_pos_out_1 = (ypoints_1 < Xcenter[0,1])
            idx = numpy.where(condition_pos_out_1 == True)[0]
            choices_pos_in = numpy.random.choice(idx, size=numpy.abs(int(gap/2)), replace=False)
            
            choices = numpy.concatenate((choices_neg_in, choices_pos_in), axis=0)
            xpoints_1 = numpy.delete(xpoints_1, choices)
            ypoints_1 = numpy.delete(ypoints_1, choices)
            class_1_ = numpy.delete(class_1_, choices)

    idx_noise = numpy.where(class_1_ == -1)[0]
    choices_noise_pos = numpy.random.choice(idx_noise, size=int(0.2 * idx_noise.size), replace=False)
    idx_noise = numpy.where(class_1_ == 1)[0]
    choices_noise_neg = numpy.random.choice(idx_noise, size=int(0.2 * idx_noise.size), replace=False)
    class_1_[choices_noise_neg], class_1_[choices_noise_pos] = -1, 1

    idx_noise = numpy.where(class_2_ == -1)[0]
    choices_noise_pos = numpy.random.choice(idx_noise, size=int(0.2 * idx_noise.size), replace=False)
    idx_noise = numpy.where(class_2_ == 1)[0]
    choices_noise_neg = numpy.random.choice(idx_noise, size=int(0.2 * idx_noise.size), replace=False)
    class_2_[choices_noise_neg], class_2_[choices_noise_pos] = -1, 1

    idx_noise = numpy.where(class_out_ == -1)[0]
    choices_noise_pos = numpy.random.choice(idx_noise, size=int(0.2 * idx_noise.size), replace=False)
    idx_noise = numpy.where(class_out_ == 1)[0]
    choices_noise_neg = numpy.random.choice(idx_noise, size=int(0.2 * idx_noise.size), replace=False)
    class_out_[choices_noise_neg], class_out_[choices_noise_pos] = -1, 1

    condition_neg_out_1 = (ypoints_1 >= Xcenter[0,1])
    condition_pos_out_1 = (ypoints_1 < Xcenter[0,1])

    condition_neg_out_2 = (ypoints_2 >= Xcenter[1,1])
    condition_pos_out_2 = (ypoints_2 < Xcenter[1,1])
    
    condition_neg_out = (ypoints_out <= 0) 
    condition_pos_out = (ypoints_out > 0)

    print(f'REGION 1 n_samples : {class_1_.size}, REGION 2 n_samples : {class_2_.size}, REGION EXT n_samples : {class_out_.size}')

    print(f'Positive class size REGION 1 : {len(class_1_[condition_pos_out_1])}, Negative class size outside region : {len(class_1_[condition_neg_out_1])}')
    print(f'Positive class size REGION 2 : {len(class_2_[condition_pos_out_2])}, Negative class size outside region : {len(class_2_[condition_neg_out_2])}')
    print(f'Positive class size REGION EXT : {len(class_out_[condition_pos_out])}, Negative class size outside region : {len(class_out_[condition_neg_out])}')

    
    # DATAFRAME
    target_name = 'class'
    data = pandas.DataFrame({'var_1': numpy.concatenate((xpoints_1, xpoints_2, xpoints_out)), 
                             'var_2': numpy.concatenate((ypoints_1, ypoints_2, ypoints_out)), 
                             'class': numpy.concatenate((class_1_, class_2_, class_out_))})
    X0 = pandas.DataFrame({'var_1': Xcenter[:,0], 'var_2': Xcenter[:,1]}, dtype=float)
    return data, target_name, X0

In [ ]:
def ConstructTargetFromRectangleGN2(xpoints, ypoints, Xcenter, BETA, lengthRectangle, widthRectangle):
    assert BETA.size == 3
    
    xpoints, ypoints = numpy.array(xpoints), numpy.array(ypoints)
    class_ = numpy.zeros(xpoints.size)

    Conditions, RAD = [], []
    for i in range(BETA.size):
        rad = numpy.sqrt((xpoints - Xcenter[i,0])**2 + (ypoints - Xcenter[i,1])**2)
        RAD.append(rad)
        Conditions.append(rad <= BETA[i])

    RAD = numpy.array(RAD)
    Conditions.append(~((RAD[0] <= BETA[0]) | (RAD[1] <= BETA[1]) | (RAD[2] <= BETA[2])))
    
    # REGION 1
    xpoints_1, ypoints_1 = xpoints[Conditions[0]], ypoints[Conditions[0]]
    class_1_ = class_[Conditions[0]]

    condition_neg_out_1 = (ypoints_1 <= 0)
    condition_pos_out_1 = (ypoints_1 > 0)
    class_1_[condition_neg_out_1], class_1_[condition_pos_out_1] = -1, 1

    # REGION 2
    xpoints_2, ypoints_2 = xpoints[Conditions[1]], ypoints[Conditions[1]]
    class_2_ = class_[Conditions[1]]

    condition_neg_out_2 = (ypoints_2 <= 0)
    condition_pos_out_2 = (ypoints_2 > 0)
    class_2_[condition_neg_out_2], class_2_[condition_pos_out_2] = -1, 1
    
    # REGION 3
    xpoints_3, ypoints_3 = xpoints[Conditions[2]], ypoints[Conditions[2]]
    class_3_ = class_[Conditions[2]]

    condition_neg_out_3 = (ypoints_3 <= 0)
    condition_pos_out_3 = (ypoints_3 > 0)
    class_3_[condition_neg_out_3], class_3_[condition_pos_out_3] = -1, 1

    # REGION EXT
    xpoints_out, ypoints_out = xpoints[Conditions[3]], ypoints[Conditions[3]]
    class_out_ = class_[Conditions[3]]
    
    condition_neg_out = (ypoints_out >= 0)
    condition_pos_out = (ypoints_out < 0)
    class_out_[condition_neg_out], class_out_[condition_pos_out] = -1, 1

    if xpoints_out[condition_neg_out].size != xpoints_out[condition_pos_out].size:
        gap = xpoints_out[condition_neg_out].size - xpoints_out[condition_pos_out].size
        idx = numpy.where(condition_neg_out == True)[0] if gap > 0 else numpy.where(condition_pos_out == True)[0]
        choices = numpy.random.choice(idx, size=numpy.abs(gap), replace=False)
        xpoints_out = numpy.delete(xpoints_out, choices)
        ypoints_out = numpy.delete(ypoints_out, choices)
        class_out_ = numpy.delete(class_out_, choices)

    # for having same size
    condition_neg_out = (ypoints_out >= 0)
    condition_neg_out_3 = (ypoints_3 <= 0)

    if xpoints_out[condition_neg_out].size != 2 * xpoints_3[condition_neg_out_3].size:
        gap = xpoints_out[condition_neg_out].size - 2 * xpoints_3[condition_neg_out_3].size
        if gap > 0 :
            # for neg out
            idx = numpy.where(condition_neg_out == True)[0]
            choices_neg_out = numpy.random.choice(idx, size=numpy.abs(gap), replace=False)
            # for pos out
            condition_pos_out = (ypoints_out < 0)
            idx = numpy.where(condition_pos_out == True)[0]
            choices_pos_out = numpy.random.choice(idx, size=numpy.abs(gap), replace=False)
            
            choices = numpy.concatenate((choices_neg_out, choices_pos_out), axis=0)
            xpoints_out = numpy.delete(xpoints_out, choices)
            ypoints_out = numpy.delete(ypoints_out, choices)
            class_out_ = numpy.delete(class_out_, choices)
        else:
            # for neg in
            idx = numpy.where(condition_neg_out_3 == True)[0]
            choices_neg_in = numpy.random.choice(idx, size=numpy.abs(int(gap/2)), replace=False)
            # for pos in
            condition_pos_out_3 = (ypoints_3 > 0)
            idx = numpy.where(condition_pos_out_3 == True)[0]
            choices_pos_in = numpy.random.choice(idx, size=numpy.abs(int(gap/2)), replace=False)
            
            choices = numpy.concatenate((choices_neg_in, choices_pos_in), axis=0)
            xpoints_3 = numpy.delete(xpoints_3, choices)
            ypoints_3 = numpy.delete(ypoints_3, choices)
            class_3_ = numpy.delete(class_3_, choices)

    condition_neg_out_1 = (ypoints_1 <= 0)
    condition_pos_out_1 = (ypoints_1 > 0)

    condition_neg_out_2 = (ypoints_2 <= 0)
    condition_pos_out_2 = (ypoints_2 > 0)
    
    condition_neg_out_3 = (ypoints_3 <= 0)
    condition_pos_out_3 = (ypoints_3 > 0)
    
    condition_neg_out = (ypoints_out >= 0)
    condition_pos_out = (ypoints_out < 0)

    print(f'REGION 1 n_samples : {class_1_.size}, REGION 2 n_samples : {class_2_.size}, REGION 3 n_samples : {class_3_.size}, REGION EXT n_samples : {class_out_.size}')

    print(f'Positive class size REGION 1 : {len(class_1_[condition_pos_out_1])}, Negative class size outside region : {len(class_1_[condition_neg_out_1])}')
    print(f'Positive class size REGION 2 : {len(class_2_[condition_pos_out_2])}, Negative class size outside region : {len(class_2_[condition_neg_out_2])}')
    print(f'Positive class size REGION 3 : {len(class_3_[condition_pos_out_3])}, Negative class size outside region : {len(class_3_[condition_neg_out_3])}')
    print(f'Positive class size REGION EXT : {len(class_out_[condition_pos_out])}, Negative class size outside region : {len(class_out_[condition_neg_out])}')

    # DATAFRAME
    target_name = 'class'
    data = pandas.DataFrame({'var_1': numpy.concatenate((xpoints_1, xpoints_2, xpoints_3, xpoints_out)), 
                             'var_2': numpy.concatenate((ypoints_1, ypoints_2, ypoints_3, ypoints_out)), 
                             'class': numpy.concatenate((class_1_, class_2_, class_3_, class_out_))})
    X0 = pandas.DataFrame({'var_1': Xcenter[:,0], 'var_2': Xcenter[:,1]}, dtype=float)
    return data, target_name, X0

In [ ]:
def DataGenerating_clf(Case=1, LengthRectangle=3, WidthRectangle=1, Ntrials=1000, lang='en'):
    numpy.random.seed(seed=0)
    assert Case == 1 or Case == 2
    assert lang == 'fr' or lang == 'en'

    if lang == 'fr':
        situation = 'Cas 1' if Case == 1 else 'Cas 2'
    else:
        situation = 'Case 1' if Case == 1 else 'Case 2'

    Xcenter = numpy.array([[-1.0,  0.], [ 1.0,  0.]]) if Case == 1 else numpy.array([[-1.4, 0.0], [0., 0.75], [1.4, 0.]])
    BETA = numpy.array([1.0, 0.8]) if Case == 1 else numpy.array([0.85, 0.4, 0.85])
    Outputs = ReplicateNTimes(UniformRandomPointInRectangle, LengthRectangle, WidthRectangle, Ntrials=Ntrials)
    data, target_name, X0 = ConstructTargetFromRectangleGN1(Outputs[0], Outputs[1], Xcenter, BETA, LengthRectangle, WidthRectangle) if Case == 1 else ConstructTargetFromRectangleGN2(Outputs[0], Outputs[1], Xcenter, BETA, LengthRectangle, WidthRectangle)
    return data, target_name, X0, BETA, situation

In [ ]:
def PlotDistribution_Synthetic_data_clf_gn(data=None, Xc=None, situation=None, learning_type_flag='Chosen', lang='fr'):
    assert lang == 'fr' or lang == 'en'
    if learning_type_flag == 'Chosen':
        assert Xc is not None
    
    columns = data.columns
    xpoints_1, xpoints_2, y = data[columns[0]], data[columns[1]], data[columns[2]]

    # figure
    if lang == 'fr':
        positive_label, negative_label, locality_label = 'Positif', 'Négatif', 'Localités'
    else:
        positive_label, negative_label, locality_label = 'Positive', 'Negative', 'Regions'
    
    pyplot.figure(figsize=(6,3), dpi=200)
    pyplot.plot(xpoints_1[y == -1], xpoints_2[y == -1], marker='_', markersize=4, color='blue', linestyle='', label=negative_label)
    pyplot.plot(xpoints_1[y == 1], xpoints_2[y == 1], marker='+', markersize=4, color='red', linestyle='', label=positive_label)
    
    if learning_type_flag == 'Chosen':
        pyplot.plot(Xc[:,0], Xc[:,1], marker='*', markersize=6, color='green', linestyle='', label=r'$\mathbf{c}$')

    # get min and max
    if numpy.min(xpoints_1) < 0:
        value = numpy.ceil(numpy.min(xpoints_1)) - 0.5
        xmin = value if value <= numpy.min(xpoints_1) else numpy.floor(numpy.min(xpoints_1))
    else:
        xmin = numpy.min(xpoints_1)
    
    if numpy.max(xpoints_1) > 0:
        value = numpy.floor(numpy.max(xpoints_1)) + 0.5
        xmax = value if value > numpy.max(xpoints_1) > 0 else numpy.ceil(numpy.max(xpoints_1))
    else:
        xmax = numpy.max(xpoints_1)
    
    if numpy.min(xpoints_2) < 0:
        value = numpy.ceil(numpy.min(xpoints_2)) - 0.5
        ymin = value if value <= numpy.min(xpoints_2) else numpy.floor(numpy.min(xpoints_2))
    else:
        ymin = numpy.min(xpoints_2)
    
    if numpy.max(xpoints_2) > 0:
        value = numpy.floor(numpy.max(xpoints_2)) + 0.5
        ymax = value if value > numpy.max(xpoints_2) > 0 else numpy.ceil(numpy.max(xpoints_2))
    else:
        ymax = numpy.max(xpoints_2)
    
    step = 0.5

    ax = pyplot.gca()
    x_locator = FixedLocator([int(value) if numpy.floor(value)==value else numpy.round(value,1) for value in numpy.arange(xmin,xmax+0.05,step)])
    y_locator = FixedLocator([int(value) if numpy.floor(value)==value else numpy.round(value,1) for value in numpy.arange(ymin,ymax+0.05,step)])
    x_formatter = FixedFormatter([str(int(value)) if numpy.floor(value)==value else str(numpy.round(value,1)) for value in numpy.arange(xmin,xmax+0.05,step)])
    y_formatter = FixedFormatter([str(int(value)) if numpy.floor(value)==value else str(numpy.round(value,1)) for value in numpy.arange(ymin,ymax+0.05,step)])
    ax.xaxis.set_major_locator(x_locator)
    ax.yaxis.set_major_locator(y_locator)
    ax.xaxis.set_major_formatter(x_formatter)
    ax.yaxis.set_major_formatter(y_formatter)

    pyplot.xlabel(r'$x_{1}$')
    pyplot.ylabel(r'$x_{2}$')
    pyplot.axis('square')
    pyplot.xlim(xmin-0.1,xmax+0.1)
    pyplot.ylim(ymin-0.1,ymax+0.1)
    pyplot.legend(loc ="upper left")

    if learning_type_flag == 'Chosen':
        pyplot.savefig(saved_img_path_clf_gn_given+'/img_synthetic_data_'+situation+'_clf_gn_given.png', dpi=500, bbox_inches='tight', transparent=True)
    else:
        pyplot.savefig(saved_img_path_clf_gn_search+'/img_synthetic_data_'+situation+'_clf_gn_search.png', dpi=500, bbox_inches='tight', transparent=True)

#### Visualization of the separating hyperplanes (for classic SVM with linear kernel)

In [ ]:
def visualize_hyperplane_gn(X=None, y=None, Xc=None, v=None, b=None, situation=None, jeu=None, lang='fr'):
    assert lang == 'fr' or lang == 'en'
    
    def get_hyperplane_value(x, v, b, offset):
        return (-v[0] * x - b + offset) / v[1]
    
    xpoints_1, xpoints_2 = X[:, 0], X[:, 1]

    # get min and max
    if numpy.min(xpoints_1) < 0:
        value = numpy.ceil(numpy.min(xpoints_1)) - 0.5
        xmin = value if value <= numpy.min(xpoints_1) else numpy.floor(numpy.min(xpoints_1))
    else:
        xmin = numpy.min(xpoints_1)
    
    if numpy.max(xpoints_1) > 0:
        value = numpy.floor(numpy.max(xpoints_1)) + 0.5
        xmax = value if value > numpy.max(xpoints_1) > 0 else numpy.ceil(numpy.max(xpoints_1))
    else:
        xmax = numpy.max(xpoints_1)
    
    if numpy.min(xpoints_2) < 0:
        value = numpy.ceil(numpy.min(xpoints_2)) - 0.5
        ymin = value if value <= numpy.min(xpoints_2) else numpy.floor(numpy.min(xpoints_2))
    else:
        ymin = numpy.min(xpoints_2)
    
    if numpy.max(xpoints_2) > 0:
        value = numpy.floor(numpy.max(xpoints_2)) + 0.5
        ymax = value if value > numpy.max(xpoints_2) > 0 else numpy.ceil(numpy.max(xpoints_2))
    else:
        ymax = numpy.max(xpoints_2)

    step = 0.5
    
    # figure
    separator_label = 'Hyperplan' if lang == 'fr' else 'Hyperplane'

    pyplot.figure(figsize=(6,3), dpi=200)
    pyplot.plot(xpoints_1[y == -1], xpoints_2[y == -1], marker='_', markersize=4, color='blue', linestyle='')
    pyplot.plot(xpoints_1[y == 1], xpoints_2[y == 1], marker='+', markersize=4, color='red', linestyle='')
    pyplot.plot(Xc[:,0], Xc[:,1], marker='*', markersize=6, color='green', linestyle='', label=r'$\mathbf{c}$')

    x0_1 = xmin
    x0_2 = xmax

    x1_1 = get_hyperplane_value(x0_1, v, b, 0)
    x1_2 = get_hyperplane_value(x0_2, v, b, 0)

    x1_1_m = get_hyperplane_value(x0_1, v, b, -1)
    x1_2_m = get_hyperplane_value(x0_2, v, b, -1)

    x1_1_p = get_hyperplane_value(x0_1, v, b, 1)
    x1_2_p = get_hyperplane_value(x0_2, v, b, 1)
    
    pyplot.plot([x0_1, x0_2], [x1_1, x1_2], color='gold', linestyle='--', label=separator_label + r' $(\mathbf{v}, b)$')
    pyplot.plot([x0_1, x0_2], [x1_1_m, x1_2_m], color='olive', linestyle='-', label='Marge')
    pyplot.plot([x0_1, x0_2], [x1_1_p, x1_2_p], color='olive', linestyle='-')
    pyplot.plot(Xc[:,0], Xc[:,1], marker='*', markersize=6, color='green', linestyle='')
    
    ax = pyplot.gca()
    x_locator = FixedLocator([int(value) if numpy.floor(value)==value else numpy.round(value,1) for value in numpy.arange(xmin,xmax+step,step)])
    y_locator = FixedLocator([int(value) if numpy.floor(value)==value else numpy.round(value,1) for value in numpy.arange(ymin,ymax+step,step)])
    x_formatter = FixedFormatter([str(int(value)) if numpy.floor(value)==value else str(numpy.round(value,1)) for value in numpy.arange(xmin,xmax+step,step)])
    y_formatter = FixedFormatter([str(int(value)) if numpy.floor(value)==value else str(numpy.round(value,1)) for value in numpy.arange(ymin,ymax+step,step)])
    ax.xaxis.set_major_locator(x_locator)
    ax.yaxis.set_major_locator(y_locator)
    ax.xaxis.set_major_formatter(x_formatter)
    ax.yaxis.set_major_formatter(y_formatter)

    pyplot.xlabel(r'$x_{1}$')
    pyplot.ylabel(r'$x_{2}$')
    pyplot.axis('square')
    pyplot.xlim(xmin-0.1,xmax+0.1)
    pyplot.ylim(ymin-0.1,ymax+0.1)
    pyplot.legend(loc="upper left")
    pyplot.savefig(saved_img_path_clf_gn_given+'/img_visualize_hyperplane_global_model_'+situation+'_'+jeu+'_clf_gn.png', dpi=500, bbox_inches='tight', transparent=True)

#### Visualization of the separating hyperplanes (for our model)

In [ ]:
def get_line_equation(x_1=None, y_1=None, x_2=None, y_2=None):
    slope = (y_1 - y_2) / (x_1 - x_2)
    bias = y_1 - slope * x_1
    return slope, bias

def get_intersection(xcenter=0, ycenter=0, radius=0.5, slope=None, bias=None):
    A = slope**2 +1
    B = 2 * (slope * (bias - ycenter) - xcenter)
    C = (bias - ycenter)**2 + xcenter**2 - radius**2

    determinant = B**2 - (4 * A * C)
    if determinant > 0 :
        X_1 = (- B - numpy.sqrt(determinant)) / (2 * A)
        X_2 = (- B + numpy.sqrt(determinant)) / (2 * A)

        Y_1 = slope * X_1 + bias
        Y_2 = slope * X_2 + bias
    else:
        X_1, X_2 = 0, 0
        Y_1, Y_2 = 0, 0
    
    return X_1, Y_1, X_2, Y_2

In [ ]:
def visualize_hybrid_hyperplane_gn(X=None, y=None, Xcenter=None, dist_type=None, Mat_dists=None, V=None, v_ext=None, B=None, b_ext=None, BETA=None, situation=None, jeu=None, option=False, learning_type_flag='Chosen', lang='fr'):
    assert dist_type == 'euclidean'
    assert BETA is not None
    assert lang == 'fr' or lang == 'en'
    
    def get_hyperplane_value(x, v, b, offset):
        return (-v[0] * x - b + offset) / v[1]
    
    xpoints_1, xpoints_2 = X[:, 0], X[:, 1]
    # get min and max
    if numpy.min(xpoints_1) < 0:
        value = numpy.ceil(numpy.min(xpoints_1)) - 0.5
        xmin = value if value <= numpy.min(xpoints_1) else numpy.floor(numpy.min(xpoints_1))
    else:
        xmin = numpy.min(xpoints_1)
    
    if numpy.max(xpoints_1) > 0:
        value = numpy.floor(numpy.max(xpoints_1)) + 0.5
        xmax = value if value > numpy.max(xpoints_1) > 0 else numpy.ceil(numpy.max(xpoints_1))
    else:
        xmax = numpy.max(xpoints_1)
    
    if numpy.min(xpoints_2) < 0:
        value = numpy.ceil(numpy.min(xpoints_2)) - 0.5
        ymin = value if value <= numpy.min(xpoints_2) else numpy.floor(numpy.min(xpoints_2))
    else:
        ymin = numpy.min(xpoints_2)
    
    if numpy.max(xpoints_2) > 0:
        value = numpy.floor(numpy.max(xpoints_2)) + 0.5
        ymax = value if value > numpy.max(xpoints_2) > 0 else numpy.ceil(numpy.max(xpoints_2))
    else:
        ymax = numpy.max(xpoints_2)
        
    step = 0.5
    step, ymin, ymax = (1, -12, 12) if option == True else (step, ymin, ymax)
    figsize = (3,12) if option == True else (6,3)
    
    # figure
    if lang == 'fr':
        positive_label, negative_label, locality_label, separator_label = 'Positif', 'Négatif', 'Localités', 'Hyperplan'
    else:
        positive_label, negative_label, locality_label, separator_label = 'Positive', 'Negative', 'Region', 'Hyperplane'

    pyplot.figure(figsize=figsize, dpi=200)
    pyplot.plot(xpoints_1[y == -1], xpoints_2[y == -1], marker='_', markersize=4, color='blue', linestyle='')
    pyplot.plot(xpoints_1[y == 1], xpoints_2[y == 1], marker='+', markersize=4, color='red', linestyle='')
    if learning_type_flag == 'Chosen':
        pyplot.plot(Xcenter[:,0], Xcenter[:,1], marker='*', markersize=6, color='green', linestyle='', label=r'$\mathbf{c}_{i}$')
    else:
        pyplot.plot(Xcenter[:,0], Xcenter[:,1], marker='*', markersize=6, color='green', linestyle='', label=r'$\mathbf{c}_{i} = \mathbf{c}_{i_{0}}$')

    # Plot beta circle
    rng = numpy.arange(0, numpy.pi *2, 0.01)
    for i in range(BETA.size):
        x_beta_circle_1 = [Xcenter[i,0] + BETA[i] * numpy.cos(v) for v in rng]
        x_beta_circle_2 = [Xcenter[i,1] + BETA[i] * numpy.sin(v) for v in rng]
        pyplot.plot(x_beta_circle_1, x_beta_circle_2, color='purple') if i != 0 else pyplot.plot(x_beta_circle_1, x_beta_circle_2, color='purple', label=locality_label + r' $\beta_{i} = k_{i}/\tau_{i}$')

    # for V = v_1 to v_n
    for i in range(BETA.size):
        xpoints_beta = xpoints_1[Mat_dists[i] <= BETA[i]]
        if xpoints_beta.size != 0:
            # get the min amd max of the absciss
            x_1_min, x_1_max = numpy.min(xpoints_beta), numpy.max(xpoints_beta)
            # projection on the separator
            y_1_min, y_1_max = get_hyperplane_value(x_1_min, V[i], B[i], 0), get_hyperplane_value(x_1_max, V[i], B[i], 0)
            y_1_min_m, y_1_max_m = get_hyperplane_value(x_1_min, V[i], B[i], -1), get_hyperplane_value(x_1_max, V[i], B[i], -1)
            y_1_min_p, y_1_max_p = get_hyperplane_value(x_1_min, V[i], B[i], 1), get_hyperplane_value(x_1_max, V[i], B[i], 1)
            # getting line equations (slope and bias)
            a_1, b_1 = get_line_equation(x_1=x_1_min, y_1=y_1_min, x_2=x_1_max, y_2=y_1_max)
            a_1_m, b_1_m = get_line_equation(x_1=x_1_min, y_1=y_1_min_m, x_2=x_1_max, y_2=y_1_max_m)
            a_1_p, b_1_p = get_line_equation(x_1=x_1_min, y_1=y_1_min_p, x_2=x_1_max, y_2=y_1_max_p)
            # get intersection between locality beta and lines
            xmin_int, ymin_int, xmax_int, ymax_int = get_intersection(xcenter=Xcenter[i,0], ycenter=Xcenter[i,1], radius=BETA[i], slope=a_1, bias=b_1)
            xmin_int_m, ymin_int_m, xmax_int_m, ymax_int_m = get_intersection(xcenter=Xcenter[i,0], ycenter=Xcenter[i,1], radius=BETA[i], slope=a_1_m, bias=b_1_m)
            xmin_int_p, ymin_int_p, xmax_int_p, ymax_int_p = get_intersection(xcenter=Xcenter[i,0], ycenter=Xcenter[i,1], radius=BETA[i], slope=a_1_p, bias=b_1_p)

            if (xmin_int == 0) & (ymin_int == 0) & (xmax_int == 0) & (ymax_int== 0):
                # Doesn't have an intersection
                pyplot.plot([x_1_min, x_1_max], [y_1_min, y_1_max], color='gold', linestyle='--') if i != 0 else pyplot.plot([x_1_min, x_1_max], [y_1_min, y_1_max], color='gold', linestyle='--', label=separator_label + r' $(\mathbf{v}_{i}, b_{i})=(\mathbf{w}_{i}, \mu_{i})$')
            else:
                pyplot.plot([xmin_int, xmax_int], [ymin_int, ymax_int], color='gold', linestyle='--') if i != 0 else pyplot.plot([xmin_int, xmax_int], [ymin_int, ymax_int], color='gold', linestyle='--', label=separator_label + r' $(\mathbf{v}_{i}, b_{i})=(\mathbf{w}_{i}, \mu_{i})$')
                
            if (xmin_int_m == 0) & (ymin_int_m == 0) & (xmax_int_m == 0) & (ymax_int_m== 0):
                # Doesn't have an intersection
                pyplot.plot([x_1_min, x_1_max], [y_1_min_m, y_1_max_m], color='olive', linestyle='-') if i != 0 else pyplot.plot([x_1_min, x_1_max], [y_1_min_m, y_1_max_m], color='olive', linestyle='-', label='Marge ' + r'$i$')
            else:
                pyplot.plot([xmin_int_m, xmax_int_m], [ymin_int_m, ymax_int_m], color='olive', linestyle='-') if i != 0 else pyplot.plot([xmin_int_m, xmax_int_m], [ymin_int_m, ymax_int_m], color='olive', linestyle='-', label='Marge ' + r'$i$')

            if (xmin_int_p == 0) & (ymin_int_p == 0) & (xmax_int_p == 0) & (ymax_int_p== 0):
                # Doesn't have an intersection
                pyplot.plot([x_1_min, x_1_max], [y_1_min_p, y_1_max_p], color='olive', linestyle='-')
            else:
                pyplot.plot([xmin_int_p, xmax_int_p], [ymin_int_p, ymax_int_p], color='olive', linestyle='-')
    
    # for v_ext = w_ext (out of beta region)
    mask = Mat_dists > BETA.reshape(BETA.size,1)
    mask = numpy.prod(mask, axis=0).astype(bool)
    xpoints_out_beta = xpoints_1[mask]
    if xpoints_out_beta.size != 0:
        # get the min amd max of the absciss
        x_1_min, x_1_max = numpy.min(xpoints_out_beta), numpy.max(xpoints_out_beta)
        # projection on the separator
        y_1_min, y_1_max = get_hyperplane_value(x_1_min, v_ext, b_ext, 0), get_hyperplane_value(x_1_max, v_ext, b_ext, 0)
        y_1_min_m, y_1_max_m = get_hyperplane_value(x_1_min, v_ext, b_ext, -1), get_hyperplane_value(x_1_max, v_ext, b_ext, -1)
        y_1_min_p, y_1_max_p = get_hyperplane_value(x_1_min, v_ext, b_ext, 1), get_hyperplane_value(x_1_max, v_ext, b_ext, 1)
        # getting line equations (slope and bias)
        a_1, b_1 = get_line_equation(x_1=x_1_min, y_1=y_1_min, x_2=x_1_max, y_2=y_1_max)
        a_1_m, b_1_m = get_line_equation(x_1=x_1_min, y_1=y_1_min_m, x_2=x_1_max, y_2=y_1_max_m)
        a_1_p, b_1_p = get_line_equation(x_1=x_1_min, y_1=y_1_min_p, x_2=x_1_max, y_2=y_1_max_p)
        
        V_int, V_int_m, V_int_p = [], [], []
        for i in range(BETA.size):
            # get intersection between locality beta and lines
            xmin_int, ymin_int, xmax_int, ymax_int = get_intersection(xcenter=Xcenter[i,0], ycenter=Xcenter[i,1], radius=BETA[i], slope=a_1, bias=b_1)
            xmin_int_m, ymin_int_m, xmax_int_m, ymax_int_m = get_intersection(xcenter=Xcenter[i,0], ycenter=Xcenter[i,1], radius=BETA[i], slope=a_1_m, bias=b_1_m)
            xmin_int_p, ymin_int_p, xmax_int_p, ymax_int_p = get_intersection(xcenter=Xcenter[i,0], ycenter=Xcenter[i,1], radius=BETA[i], slope=a_1_p, bias=b_1_p)          
            
            if (xmin_int != 0) & (ymin_int != 0) & (xmax_int != 0) & (ymax_int!= 0):
                V_int.append((xmin_int, ymin_int, xmax_int, ymax_int))
                V_int_m.append((xmin_int_m, ymin_int_m, xmax_int_m, ymax_int_m))
                V_int_p.append((xmin_int_p, ymin_int_p, xmax_int_p, ymax_int_p))

        V_int, V_int_m, V_int_p = numpy.array(V_int), numpy.array(V_int_m), numpy.array(V_int_p)
        
        if (len(V_int) == 0) and (len(V_int_m) == 0) and (len(V_int_p) == 0):
            pyplot.plot([x_1_min, x_1_max], [y_1_min, y_1_max], color='maroon', linestyle='--', label=separator_label + r' $(\mathbf{v}_{ext}, b_{ext})=(\mathbf{w}_{ext}, \mu_{ext})$')
            pyplot.plot([x_1_min, x_1_max], [y_1_min_m, y_1_max_m], color='grey', linestyle='-', label='Marge ' + r'$ext$')
            pyplot.plot([x_1_min, x_1_max], [y_1_min_p, y_1_max_p], color='grey', linestyle='-')
        else:
            V_int = V_int[V_int[:, 0].argsort()]
            V_int_m = V_int_m[V_int_m[:, 0].argsort()]
            V_int_p = V_int_p[V_int_p[:, 0].argsort()]
    
            length = V_int.shape[0]
            in_idx = []
            
            if length > 1:
                for i in range(length - 1):
                    if V_int[i,2] > V_int[i+1,2]:
                        in_idx.append(i+1)

            # Supprime les regions inclues dans d'autres
            V_int, V_int_m, V_int_p = numpy.delete(V_int, in_idx, axis=0), numpy.delete(V_int_m, in_idx, axis=0), numpy.delete(V_int_p, in_idx, axis=0)
            length = V_int.shape[0]
                        
            if length == 1 :
                if (V_int[0,0] != 0) & (V_int[0,1] != 0) & (V_int[0,2] != 0) & (V_int[0,3]!= 0):
                    pyplot.plot([x_1_min, V_int[0,0]], [y_1_min, V_int[0,1]], color='maroon', linestyle='--', label=separator_label + r' $(\mathbf{v}_{ext}, b_{ext})=(\mathbf{w}_{ext}, \mu_{ext})$')
                    pyplot.plot([V_int[0,2], x_1_max], [V_int[0,3], y_1_max], color='maroon', linestyle='--')
                else :
                    pyplot.plot([x_1_min, x_1_max], [y_1_min, y_1_max], color='maroon', linestyle='--', label=separator_label + r' $(\mathbf{v}_{ext}, b_{ext})=(\mathbf{w}_{ext}, \mu_{ext})$')

                if (V_int_m[0,0] != 0) & (V_int_m[0,1] != 0) & (V_int_m[0,2] != 0) & (V_int_m[0,3]!= 0):
                    pyplot.plot([x_1_min, V_int_m[0,0]], [y_1_min_m, V_int_m[0,1]], color='grey', linestyle='-', label='Marge ' + r'$ext$')
                    pyplot.plot([V_int_m[0,2], x_1_max], [V_int_m[0,3], y_1_max_m], color='grey', linestyle='-')
                else :
                    pyplot.plot([x_1_min, x_1_max], [y_1_min_m, y_1_max_m], color='grey', linestyle='-', label='Marge ' + r'$ext$')

                if (V_int_p[0,0] != 0) & (V_int_p[0,1] != 0) & (V_int_p[0,2] != 0) & (V_int_p[0,3]!= 0):
                    pyplot.plot([x_1_min, V_int_p[0,0]], [y_1_min_p, V_int_p[0,1]], color='grey', linestyle='-')
                    pyplot.plot([V_int_p[0,2], x_1_max], [V_int_p[0,3], y_1_max_p], color='grey', linestyle='-')
                else :
                    pyplot.plot([x_1_min, x_1_max], [y_1_min_p, y_1_max_p], color='grey', linestyle='-')
        
            elif length == 2 :
                condition_1 = (V_int[0,0] != 0) & (V_int[0,1] != 0) & (V_int[0,2] != 0) & (V_int[0,3]!= 0)
                condition_2 = (V_int[1,0] != 0) & (V_int[1,1] != 0) & (V_int[1,2] != 0) & (V_int[1,3]!= 0)
                if condition_1 and condition_2 :
                    pyplot.plot([x_1_min, V_int[0,0]], [y_1_min, V_int[0,1]], color='maroon', linestyle='--', label=separator_label + r' $(\mathbf{v}_{ext}, b_{ext})=(\mathbf{w}_{ext}, \mu_{ext})$')
                    pyplot.plot([V_int[0,2], V_int[1,0]], [V_int[0,3], V_int[1,1]], color='maroon', linestyle='--')
                    pyplot.plot([V_int[1,2], x_1_max], [V_int[1,3], y_1_max], color='maroon', linestyle='--')
                elif condition_1 :
                    pyplot.plot([x_1_min, V_int[0,0]], [y_1_min, V_int[0,1]], color='maroon', linestyle='--', label=separator_label + r' $(\mathbf{v}_{ext}, b_{ext})=(\mathbf{w}_{ext}, \mu_{ext})$')
                    pyplot.plot([V_int[0,2], x_1_max], [V_int[0,3], y_1_max], color='maroon', linestyle='--')
                elif condition_2 :
                    pyplot.plot([x_1_min, V_int[1,0]], [y_1_min, V_int[1,1]], color='maroon', linestyle='--', label=separator_label + r' $(\mathbf{v}_{ext}, b_{ext})=(\mathbf{w}_{ext}, \mu_{ext})$')
                    pyplot.plot([V_int[1,2], x_1_max], [V_int[1,3], y_1_max], color='maroon', linestyle='--')
                else :
                    pyplot.plot([x_1_min, x_1_max], [y_1_min, y_1_max], color='maroon', linestyle='--', label=separator_label + r' $(\mathbf{v}_{ext}, b_{ext})=(\mathbf{w}_{ext}, \mu_{ext})$')

                condition_1 = (V_int_m[0,0] != 0) & (V_int_m[0,1] != 0) & (V_int_m[0,2] != 0) & (V_int_m[0,3]!= 0)
                condition_2 = (V_int_m[1,0] != 0) & (V_int_m[1,1] != 0) & (V_int_m[1,2] != 0) & (V_int_m[1,3]!= 0)
                if condition_1 and condition_2 :
                    pyplot.plot([x_1_min, V_int_m[0,0]], [y_1_min_m, V_int_m[0,1]], color='grey', linestyle='-', label='Marge ' + r'$ext$')
                    pyplot.plot([V_int_m[0,2], V_int_m[1,0]], [V_int_m[0,3], V_int_m[1,1]], color='grey', linestyle='-')
                    pyplot.plot([V_int_m[1,2], x_1_max], [V_int_m[1,3], y_1_max_m], color='grey', linestyle='-')
                elif condition_1 :
                    pyplot.plot([x_1_min, V_int_m[0,0]], [y_1_min_m, V_int_m[0,1]], color='grey', linestyle='-', label='Marge ' + r'$ext$')
                    pyplot.plot([V_int_m[0,2], x_1_max], [V_int_m[0,3], y_1_max_m], color='grey', linestyle='-')
                elif condition_2 :
                    pyplot.plot([x_1_min, V_int_m[1,0]], [y_1_min_m, V_int_m[1,1]], color='grey', linestyle='-', label='Marge ' + r'$ext$')
                    pyplot.plot([V_int_m[1,2], x_1_max], [V_int_m[1,3], y_1_max_m], color='grey', linestyle='-')
                else :
                    pyplot.plot([x_1_min, x_1_max], [y_1_min_m, y_1_max_m], color='grey', linestyle='-', label='Marge ' + r'$ext$')

                condition_1 = (V_int_p[0,0] != 0) & (V_int_p[0,1] != 0) & (V_int_p[0,2] != 0) & (V_int_p[0,3]!= 0)
                condition_2 = (V_int_p[1,0] != 0) & (V_int_p[1,1] != 0) & (V_int_p[1,2] != 0) & (V_int_p[1,3]!= 0)
                if condition_1 and condition_2 :
                    pyplot.plot([x_1_min, V_int_p[0,0]], [y_1_min_p, V_int_p[0,1]], color='grey', linestyle='-')
                    pyplot.plot([V_int_p[0,2], V_int_p[1,0]], [V_int_p[0,3], V_int_p[1,1]], color='grey', linestyle='-')
                    pyplot.plot([V_int_p[1,2], x_1_max], [V_int_p[1,3], y_1_max_p], color='grey', linestyle='-')
                elif condition_1 :
                    pyplot.plot([x_1_min, V_int_p[0,0]], [y_1_min_p, V_int_p[0,1]], color='grey', linestyle='-')
                    pyplot.plot([V_int_p[0,2], x_1_max], [V_int_p[0,3], y_1_max_p], color='grey', linestyle='-')
                elif condition_2 :
                    pyplot.plot([x_1_min, V_int_p[1,0]], [y_1_min_p, V_int_p[1,1]], color='grey', linestyle='-')
                    pyplot.plot([V_int_p[1,2], x_1_max], [V_int_p[1,3], y_1_max_p], color='grey', linestyle='-')
                else :
                    pyplot.plot([x_1_min, x_1_max], [y_1_min_p, y_1_max_p], color='grey', linestyle='-')
            
            elif length == 3 :
                pyplot.plot([x_1_min, V_int[0,0]], [y_1_min, V_int[0,1]], color='maroon', linestyle='--', label=separator_label + r' $(\mathbf{v}_{ext}, b_{ext})=(\mathbf{w}_{ext}, \mu_{ext})$')
                pyplot.plot([V_int[0,2], V_int[1,0]], [V_int[0,3], V_int[1,1]], color='maroon', linestyle='--')
                pyplot.plot([V_int[1,2], V_int[2,0]], [V_int[1,3], V_int[2,1]], color='maroon', linestyle='--')
                pyplot.plot([V_int[2,2], x_1_max], [V_int[2,3], y_1_max], color='maroon', linestyle='--')
    
                pyplot.plot([x_1_min, V_int_m[0,0]], [y_1_min_m, V_int_m[0,1]], color='grey', linestyle='-', label='Marge ' + r'$ext$')
                pyplot.plot([V_int_m[0,2], V_int_m[1,0]], [V_int_m[0,3], V_int_m[1,1]], color='grey', linestyle='-')
                pyplot.plot([V_int_m[1,2], V_int_m[2,0]], [V_int_m[1,3], V_int_m[2,1]], color='grey', linestyle='-')
                pyplot.plot([V_int_m[2,2], x_1_max], [V_int_m[2,3], y_1_max_m], color='grey', linestyle='-')
    
                pyplot.plot([x_1_min, V_int_p[0,0]], [y_1_min_p, V_int_p[0,1]], color='grey', linestyle='-')
                pyplot.plot([V_int_p[0,2], V_int_p[1,0]], [V_int_p[0,3], V_int_p[1,1]], color='grey', linestyle='-')
                pyplot.plot([V_int_p[1,2], V_int_p[2,0]], [V_int_p[1,3], V_int_p[2,1]], color='grey', linestyle='-')
                pyplot.plot([V_int_p[2,2], x_1_max], [V_int_p[2,3], y_1_max_p], color='grey', linestyle='-')
        
    pyplot.plot(Xcenter[:,0], Xcenter[:,1], marker='*', markersize=6, color='green', linestyle='')
    
    ax = pyplot.gca()
    x_locator = FixedLocator([int(value) if numpy.floor(value)==value else numpy.round(value,1) for value in numpy.arange(xmin,xmax+step,step)])
    y_locator = FixedLocator([int(value) if numpy.floor(value)==value else numpy.round(value,1) for value in numpy.arange(ymin, ymax+step,step)])
    x_formatter = FixedFormatter([str(int(value)) if numpy.floor(value)==value else str(numpy.round(value,1)) for value in numpy.arange(xmin,xmax+step,step)])
    y_formatter = FixedFormatter([str(int(value)) if numpy.floor(value)==value else str(numpy.round(value,1)) for value in numpy.arange(ymin,ymax+step,step)])
    ax.xaxis.set_major_locator(x_locator)
    ax.yaxis.set_major_locator(y_locator)
    ax.xaxis.set_major_formatter(x_formatter)
    ax.yaxis.set_major_formatter(y_formatter)
    
    pyplot.xlabel(r'$x_{1}$')
    pyplot.ylabel(r'$x_{2}$')
    pyplot.axis('square')
    pyplot.xlim(xmin-0.1,xmax+0.1)
    pyplot.ylim(ymin-0.1,ymax+0.1)
    #pyplot.legend(ncol=3, loc ="upper left", bbox_to_anchor=(0, 0.51))

    if learning_type_flag == 'Chosen':
        pyplot.savefig(saved_img_path_clf_gn_given+'/img_visualize_hyperplane_hybrid_model_'+situation+'_'+jeu+'_clf_gn_given.png', dpi=1000, bbox_inches='tight', transparent=True)
    else:
        pyplot.savefig(saved_img_path_clf_gn_search+'/img_visualize_hyperplane_hybrid_model_'+situation+'_'+jeu+'_clf_gn_search.png', dpi=1000, bbox_inches='tight', transparent=True)


    figsize = (1,6)
    pyplot.figure(figsize=figsize, dpi=200)
    ax_leg = pyplot.gca()
    ax_leg.legend(*ax.get_legend_handles_labels(), ncol=3, loc='center', fontsize=9, facecolor='white', edgecolor='black', framealpha=0.05)
    ax_leg.axis('off')
    if learning_type_flag == 'Chosen':
        pyplot.savefig(saved_img_path_clf_gn_given+'/img_MoTLE_legend_clf_gn_given.png', dpi=1000, bbox_inches='tight', transparent=True)
    else:
        pyplot.savefig(saved_img_path_clf_gn_search+'/img_MoTLE_legend_clf_gn_search.png', dpi=1000, bbox_inches='tight', transparent=True)    

## 4. Linear Regression

#### Generate synthetic data

In [ ]:
def ConstructTargetFromLinearRegressionGN1(X, y, beta=1):
    X = X.reshape(X.shape[0],)
    X_sym, y_sym = -X, y
    step = 0.15
    
    # create data inside and outside beta region
    condition = (X >= -beta) & (X <= beta)
    X_1, y_1 = X[condition], y[condition]
    x0_1, y0_1 = numpy.mean(X_1), numpy.mean(y_1)

    # create data outside beta region
    condition_2 = (X_sym >= -beta) & (X_sym <= beta)
    X_out, y_out = X_sym[condition_2], y_sym[condition_2]
    Slope_1 = numpy.mean(y_out[X_out <= 0] / X_out[X_out <= 0]) + numpy.random.randn(y_out[X_out <= 0].size)
    Slope_2 = numpy.mean(y_out[X_out > 0] / X_out[X_out > 0]) + numpy.random.randn(y_out[X_out > 0].size)
    
    X_out[X_out <= 0] = X_out[X_out <= 0] - (step + beta)
    X_out[X_out > 0] = X_out[X_out > 0] + (step + beta)

    y_out[X_out <= 0] = Slope_1 * X_out[X_out <= 0]
    y_out[X_out > 0] = Slope_2 * X_out[X_out > 0]

    # concatenate data
    X0, Y0 = numpy.array([x0_1]), numpy.array([y0_1])
    X = numpy.concatenate((X_1, X_out))
    y = numpy.concatenate((y_1, y_out))

    # data scaling
    numeric_vars, category_vars = data_analysis.num_and_cat_features(pandas.DataFrame(X), print_var = False)
    X, X0 = data_analysis.data_scaling(scale_type='Standardscaler', numeric_vars=numeric_vars, xtrain=pandas.DataFrame(X), data_1=pandas.DataFrame(X0), data_2=None, data_3=None)
    X, X0 = X.values, X0.values
    y, Y0 = data_analysis.data_scaling(scale_type='MaxAbsScaler', numeric_vars=numeric_vars, xtrain=pandas.DataFrame(y), data_1=pandas.DataFrame(Y0), data_2=None, data_3=None)
    y, Y0 = y.values, Y0.values
    
    X0, X, y = X0.reshape(X0.size), X.reshape(X.size), y.reshape(y.size)
    cond = (X >= -2*beta) & (X <= 2*beta)
    X, y = X[cond], y[cond]

    condition_1 = abs(X - X0[0])<= beta
    condition_2 = ~condition_1
    print(f'REGION 1 n_samples : {X[condition_1].size}, REGION EXT n_samples : {X[condition_2].size}')
    
    # Make dataframe
    target_name = 'target'
    data = pandas.DataFrame({'var': X, 'target': y})
    X0 = pandas.DataFrame({'var': X0}, dtype=float)
    BETA = numpy.array([beta])

    return data, target_name, X0, Y0, BETA

In [ ]:
def ConstructTargetFromLinearRegressionGN2(X, y, beta=1, const=3):
    X = X.reshape(X.shape[0],)
    X_sym, y_sym = -X, y
    step = 0.15
    
    # create data inside the region 1
    condition = (X >= -beta) & (X <= beta)
    X_1, y_1 = X[condition], y[condition]
    x0_1, y0_1 = numpy.mean(X_1), numpy.mean(y_1)
    
    # create data inside the region 2
    condition_sym = (X_sym >= -beta) & (X_sym <= beta)
    X_2, y_2 = X_sym[condition_sym] + 2 * (beta + step), y_sym[condition_sym]
    x0_2, y0_2 = numpy.mean(X_2), numpy.mean(y_2)
    
    # create data outside beta region
    choices = numpy.random.choice(numpy.arange(X_1.size), size=int(0.5 * X_1.size), replace=False)
    X_out_l, y_out_l = X_1[choices] - 2 * (beta + step), numpy.ones(choices.size) * const + numpy.random.randn(choices.size)
    X_out_r, y_out_r = X_2[choices] + 2 * (beta + step), numpy.ones(choices.size) * const + numpy.random.randn(choices.size)

    # concatenate data
    X0, Y0 = numpy.array([x0_1, x0_2]), numpy.array([y0_1, y0_2])
    X = numpy.concatenate((X_1, X_2, X_out_l, X_out_r))
    y = numpy.concatenate((y_1, y_2, y_out_l, y_out_r))

    # data scaling
    numeric_vars, category_vars = data_analysis.num_and_cat_features(pandas.DataFrame(X), print_var = False)
    X, X0 = data_analysis.data_scaling(scale_type='Standardscaler', numeric_vars=numeric_vars, xtrain=pandas.DataFrame(X), data_1=pandas.DataFrame(X0), data_2=None, data_3=None)
    X, X0 = X.values, X0.values
    y, Y0 = data_analysis.data_scaling(scale_type='MaxAbsScaler', numeric_vars=numeric_vars, xtrain=pandas.DataFrame(y), data_1=pandas.DataFrame(Y0), data_2=None, data_3=None)
    y, Y0 = y.values, Y0.values
    
    X0, X, y = X0.reshape(X0.size), X.reshape(X.size), y.reshape(y.size)
    beta = (abs(X0[0] - X0[1])) / 2

    condition_1 = abs(X - X0[0])<= beta
    condition_2 = abs(X - X0[1])<= beta
    condition_3 = ~(condition_1 | condition_2)
    print(f'REGION 1 n_samples : {X[condition_1].size}, REGION 2 n_samples : {X[condition_2].size}, REGION EXT n_samples : {X[condition_3].size}')
     
    # Make dataframe
    target_name = 'target'
    data = pandas.DataFrame({'var': X, 'target': y})
    X0 = pandas.DataFrame({'var': X0}, dtype=float)
    BETA = numpy.array([beta, beta])

    return data, target_name, X0, Y0, BETA

In [ ]:
def DataGenerating_reg(Case=1, Ntrials=2000, lang='en'):
    numpy.random.seed(seed=0)
    assert Case == 1 or Case == 2
    assert lang == 'fr' or lang == 'en'

    if lang == 'fr':
        situation = 'Cas 1' if Case == 1 else 'Cas 2'
    else:
        situation = 'Case 1' if Case == 1 else 'Case 2'

    X, y = make_regression(n_samples=Ntrials, n_features=1, noise=2.0, bias=0.0, random_state=42)
    data, target_name, x0, y0, beta = ConstructTargetFromLinearRegressionGN1(X, y, beta=1.0) if Case == 1 else ConstructTargetFromLinearRegressionGN2(X, y, beta=1, const=3)
    return data, target_name, x0, y0, beta, situation

In [ ]:
def PlotDistribution_Synthetic_data_reg_gn(data=None, Xc=None, Yc=None, situation=None, learning_type_flag='Chosen', lang='fr'):
    assert lang == 'fr' or lang == 'en'
    if learning_type_flag == 'Chosen':
        assert Xc is not None
        assert Yc is not None
        
    columns = data.columns
    xpoints, y = data[columns[0]], data[columns[1]]

    xmin, xmax = -max(round(abs(xpoints))), max(round(abs(xpoints)))
    ymin, ymax = -max(abs(y)), max(abs(y))
    x_std = (numpy.linspace(xmin,xmax,5)[1] - numpy.linspace(xmin,xmax,5)[0]) / 10
    y_std = (numpy.linspace(ymin,ymax,5)[1] - numpy.linspace(ymin,ymax,5)[0]) / 10

    # figure
    instances_label, locality_label = ('Données', 'Localités') if lang == 'fr' else ('Dataset', 'Regions')

    pyplot.figure(figsize=(4.5,3), dpi=200)
    pyplot.plot(xpoints, y, marker='o', markersize=3.0, color='blue', linestyle='', label=instances_label)
    
    if learning_type_flag == 'Chosen':
        pyplot.plot(Xc, Yc, marker='*', markersize=4.75, color='green', linestyle='', label=r'$\mathbf{c}$')

    ax = pyplot.gca()
    x_locator = FixedLocator([int(value) if numpy.floor(value)==value else numpy.round(value,1) for value in numpy.linspace(xmin,xmax,9)])
    y_locator = FixedLocator([int(value) if numpy.floor(value)==value else numpy.round(value,2) for value in numpy.linspace(ymin,ymax,5)])
    x_formatter = FixedFormatter([str(int(value)) if numpy.floor(value)==value else str(numpy.round(value,1)) for value in numpy.linspace(xmin,xmax,9)])
    y_formatter = FixedFormatter([str(int(value)) if numpy.floor(value)==value else str(numpy.round(value,2)) for value in numpy.linspace(ymin,ymax,5)])
    ax.xaxis.set_major_locator(x_locator)
    ax.yaxis.set_major_locator(y_locator)
    ax.xaxis.set_major_formatter(x_formatter)
    ax.yaxis.set_major_formatter(y_formatter)

    pyplot.xlabel(r'$x$')
    pyplot.ylabel(r'$y$')
    pyplot.axis('square')
    pyplot.xlim(xmin-x_std, xmax+x_std)
    pyplot.ylim(ymin-y_std, ymax+y_std)
    pyplot.legend(loc ="upper right")
    
    if learning_type_flag == 'Chosen':
        pyplot.savefig(saved_img_path_reg_gn_given+'/img_synthetic_data_'+situation+'_reg_gn_given.png', dpi=500, bbox_inches='tight', transparent=True)
    else:
        pyplot.savefig(saved_img_path_reg_gn_search+'/img_synthetic_data_'+situation+'_reg_gn_search.png', dpi=500, bbox_inches='tight', transparent=True)

#### Visualization of the separating hyperplanes (for classic SVR with linear kernel)

In [ ]:
def get_regression_line_value(x, w, mu):
    return numpy.dot(x, w) + mu

def get_xlimit_ylimit(X=None, y=None, w=None, b=None): 
    y_pred = get_regression_line_value(X, w, b)

    xmin, xmax = -numpy.round(numpy.ceil(max(abs(X)))), numpy.round(numpy.ceil(max(abs(X))))
    ymin, ymax = -max(max(abs(y)), max(abs(y_pred))), max(max(abs(y)), max(abs(y_pred)))
    
    return xmin, xmax, ymin, ymax

def visualize_lr_reg_gn(X=None, y=None, Xc=None, Yc=None, w=None, b=None, xmin=None, xmax=None, ymin=None, ymax=None, situation=None, jeu=None, lang='fr'):
    assert lang == 'fr' or lang == 'en'
    
    y_pred = get_regression_line_value(X, w, b)
    
    x_std = (numpy.linspace(xmin,xmax,5)[1] - numpy.linspace(xmin,xmax,5)[0]) / 10
    y_std = (numpy.linspace(ymin,ymax,5)[1] - numpy.linspace(ymin,ymax,5)[0]) / 10
    
    # figure
    if jeu == 'Apprentissage' :
        jeu_label = 'Jeu d\'entraînement' if lang == 'fr' else 'Training set'
    else :
        jeu_label = 'Jeu de test' if lang == 'fr' else 'Testing set'
        
    regression_line_label = 'Droite de régression' if lang == 'fr' else 'Regression line'
    
    pyplot.figure(figsize=(4.5,3), dpi=200)
    pyplot.plot(X, y, marker='o', markersize=3.0, color='blue', linestyle='', label=jeu_label)
    pyplot.plot(Xc, Yc, marker='*', markersize=4.75, color='green', linestyle='', label=r'$\mathbf{c}$')
    pyplot.plot(X, y_pred, color='gold', linestyle='-', label=regression_line_label + r' $(\mathbf{v}, b)$')
    pyplot.plot(Xc, Yc, marker='*', markersize=4.75, color='green', linestyle='')
    
    ax = pyplot.gca()
    x_locator = FixedLocator([int(value) if numpy.floor(value)==value else float(numpy.round(value,1)) for value in numpy.linspace(xmin,xmax,9)])
    y_locator = FixedLocator([int(value) if numpy.floor(value)==value else float(numpy.round(value,2)) for value in numpy.linspace(ymin,ymax,5)])
    x_formatter = FixedFormatter([str(int(value)) if numpy.floor(value)==value else str(float(numpy.round(value,1))) for value in numpy.linspace(xmin,xmax,9)])
    y_formatter = FixedFormatter([str(int(value)) if numpy.floor(value)==value else str(float(numpy.round(value,2))) for value in numpy.linspace(ymin,ymax,5)])
    ax.xaxis.set_major_locator(x_locator)
    ax.yaxis.set_major_locator(y_locator)
    ax.xaxis.set_major_formatter(x_formatter)
    ax.yaxis.set_major_formatter(y_formatter)
    
    pyplot.xlabel(r'$x$')
    pyplot.ylabel(r'$y$')
    pyplot.axis('square')
    pyplot.xlim(xmin-x_std, xmax+x_std)
    pyplot.ylim(ymin-y_std, ymax+y_std)
    pyplot.legend(loc ="upper right")
    
    pyplot.savefig(saved_img_path_reg_gn_given+'/img_visualize_hyperplane_global_model_'+situation+'_'+jeu+'_reg_gn.png', dpi=500, bbox_inches='tight', transparent=True)

#### Visualization of the separating hyperplanes (for our model)

In [ ]:
def get_xlimit_ylimit_hybrid_gn(X=None, y=None, Xc=None, dist_type=None, Mat_dists=None, V=None, v_ext=None, B=None, b_ext=None, BETA=None):
    assert dist_type == 'euclidean'
    assert BETA is not None

    # for V = v_1 to v_n
    y_pred_beta = []
    for i in range(BETA.size):
        xpoints_beta, y_beta = X[Mat_dists[i] <= BETA[i]], y[Mat_dists[i] <= BETA[i]]
        if xpoints_beta.size != 0:
            y_pred_beta = numpy.concatenate((y_pred_beta, get_regression_line_value(xpoints_beta, V[i], B[i])))
        
    # for v_ext = w_ext (out of beta region)
    mask = Mat_dists > BETA.reshape(BETA.size,1)
    mask = numpy.prod(mask, axis=0).astype(bool)
    xpoints_out_beta, y_out_beta = X[mask], y[mask]
    if xpoints_out_beta.size != 0:
        y_pred_out_beta = get_regression_line_value(xpoints_out_beta, v_ext, b_ext)
        
    X = X.reshape(X.shape[0])
    xmax = numpy.round(numpy.ceil(max(max(abs(X)), abs(Xc[numpy.argmin(Xc)] - BETA[numpy.argmin(Xc)]), abs(Xc[numpy.argmax(Xc)] + BETA[numpy.argmax(Xc)]))))
    xmin = -xmax
    
    if (y_pred_beta.size != 0) and (xpoints_out_beta.size != 0) :
        ymax = max(max(abs(y)), max(abs(y_pred_beta)), max(abs(y_pred_out_beta)))
        ymin = -ymax
    elif (y_pred_beta.size != 0):
        ymax = max(max(abs(y)), max(abs(y_pred_beta)))
        ymin = -ymax
    else:
        ymax = max(max(abs(y)), max(abs(y_pred_out_beta)))
        ymin = -ymax
    
    return xmin, xmax, ymin, ymax

In [ ]:
def visualize_hybrid_lr_reg_gn(X=None, y=None, Xc=None, Yc=None, dist_type=None, Mat_dists=None, V=None, v_ext=None, B=None, b_ext=None, BETA=None, xmin=None, xmax=None, ymin=None, ymax=None, situation=None, jeu=None, learning_type_flag='Chosen', lang='fr'):
    assert dist_type == 'euclidean'
    assert BETA is not None
    assert lang == 'fr' or lang == 'en'
    
    Xc = Xc.reshape(Xc.shape[0])
    idx = numpy.argsort(Xc)
    Xc, Yc, V, B, BETA = Xc[idx], Yc[idx], V[idx], B[idx], BETA[idx]
    Xc = Xc.reshape(Xc.shape[0], 1)
    Mat_dists = Mat_dists[idx]

    # figure
    if jeu == 'Apprentissage' :
        jeu_label = 'Jeu d\'entraînement' if lang == 'fr' else 'Training set'
    else :
        jeu_label = 'Jeu de test' if lang == 'fr' else 'Testing set'
        
    locality_label = 'Localités' if lang == 'fr' else 'Regions'
    regression_line_label = 'Droite de régression' if lang == 'fr' else 'Regression line'
    
    pyplot.figure(figsize=(4.5,3), dpi=200)
    pyplot.plot(X, y, marker='o', markersize=3.0, color='blue', linestyle='')
    if learning_type_flag == 'Chosen':
        pyplot.plot(Xc, Yc, marker='*', markersize=4.75, color='green', linestyle='', label=r'$\mathbf{c}_{i}$')
    else:
        pyplot.plot(Xc, Yc, marker='*', markersize=4.75, color='green', linestyle='', label=r'$\mathbf{c}_{i} = \mathbf{c}_{i_{0}}$')

    x_std = (numpy.linspace(xmin,xmax,5)[1] - numpy.linspace(xmin,xmax,5)[0]) / 10
    y_std = (numpy.linspace(ymin,ymax,5)[1] - numpy.linspace(ymin,ymax,5)[0]) / 10

    for i in range(BETA.size):
        # Plot inner region
        x_v_1 = (Yc[i] - (y_std))/(ymax - ymin + 2 * y_std) + (1 - ((ymax + y_std)/(ymax - ymin + 2 * y_std)))
        x_v_2 = (Yc[i] + (y_std))/(ymax - ymin + 2 * y_std) + (1 - ((ymax + y_std)/(ymax - ymin + 2 * y_std)))

        x_h_1 = (Xc[i] - BETA[i])/(xmax - xmin + 2 * x_std) + (1 - ((xmax + x_std)/(xmax - xmin + 2 * x_std)))
        x_h_2 = (Xc[i] + BETA[i])/(xmax - xmin + 2 * x_std) + (1 - ((xmax + x_std)/(xmax - xmin + 2 * x_std)))

        pyplot.axhline(y = Yc[i], xmin=x_h_1, xmax=x_h_2, color = 'purple')
        pyplot.axvline(x = Xc[i] - BETA[i], ymin=x_v_1, ymax=x_v_2, color = 'purple') if i != 0 else pyplot.axvline(x = Xc[i] - BETA[i], ymin=x_v_1, ymax=x_v_2, color = 'purple', label=locality_label + r' $\beta_{i} = k_{i}/\tau_{i}$')
        pyplot.axvline(x = Xc[i] + BETA[i], ymin=x_v_1, ymax=x_v_2, color = 'purple')
    
    # for V = v_1 to v_n
    for i in range(BETA.size):
        xpoints_beta, y_beta = X[Mat_dists[i] <= BETA[i]], y[Mat_dists[i] <= BETA[i]]   
        if xpoints_beta.size != 0:
            y_pred_beta = get_regression_line_value(xpoints_beta, V[i], B[i])
            pyplot.plot(xpoints_beta, y_pred_beta, color='gold', linestyle='-') if i != 0 else pyplot.plot(xpoints_beta, y_pred_beta, color='gold', linestyle='-', label=regression_line_label + r' $(\mathbf{v}_{i}, b_{i})=(\mathbf{w}_{i}, \mu_{i})$')
              
    # for v_ext = w_ext (out of beta region)
    mask = Mat_dists > BETA.reshape(BETA.size,1)
    mask = numpy.prod(mask, axis=0).astype(bool)
    xpoints_out_beta, y_out_beta = X[mask], y[mask]
    if xpoints_out_beta.size != 0:
        y_pred_out_beta = get_regression_line_value(xpoints_out_beta, v_ext, b_ext)
        xpoints_out_beta = xpoints_out_beta.reshape(-1)
        
        cond_low = xpoints_out_beta < (Xc[numpy.argmin(Xc)] - BETA[numpy.argmin(Xc)])
        cond_high = xpoints_out_beta > (Xc[numpy.argmax(Xc)] + BETA[numpy.argmax(Xc)])

        plot_label = False
        if cond_low.sum() != 0 or cond_high.sum() != 0:
            plot_label = True
            pyplot.plot(xpoints_out_beta[cond_low], y_pred_out_beta[cond_low], color='indigo', linestyle='-', label=regression_line_label + r' $(\mathbf{v}_{ext}, b_{ext})=(\mathbf{w}_{ext}, \mu_{ext})$')
            pyplot.plot(xpoints_out_beta[cond_high], y_pred_out_beta[cond_high], color='indigo', linestyle='-')

        for i in range(Xc.shape[0]-1):
            cond_between = (xpoints_out_beta > (Xc[i] + BETA[i])) & (xpoints_out_beta < (Xc[i+1] - BETA[i+1]))
            if plot_label == False and i == 0 :
                pyplot.plot(xpoints_out_beta[cond_between], y_pred_out_beta[cond_between], color='indigo', linestyle='-', label=regression_line_label + r' $(\mathbf{v}_{ext}, b_{ext})=(\mathbf{w}_{ext}, \mu_{ext})$')
            else:
                pyplot.plot(xpoints_out_beta[cond_between], y_pred_out_beta[cond_between], color='indigo', linestyle='-')

    pyplot.plot(Xc, Yc, marker='*', markersize=4.75, color='green', linestyle='')
    
    ax = pyplot.gca()
    x_locator = FixedLocator([int(value) if numpy.floor(value)==value else float(numpy.round(value,1)) for value in numpy.linspace(xmin,xmax,9)]) #numpy.arange(xmin,xmax+1)
    y_locator = FixedLocator([int(value) if numpy.floor(value)==value else float(numpy.round(value,2)) for value in numpy.linspace(ymin,ymax,5)])
    x_formatter = FixedFormatter([str(int(value)) if numpy.floor(value)==value else str(float(numpy.round(value,1))) for value in numpy.linspace(xmin,xmax,9)])
    y_formatter = FixedFormatter([str(int(value)) if numpy.floor(value)==value else str(float(numpy.round(value,2))) for value in numpy.linspace(ymin,ymax,5)])
    ax.xaxis.set_major_locator(x_locator)
    ax.yaxis.set_major_locator(y_locator)
    ax.xaxis.set_major_formatter(x_formatter)
    ax.yaxis.set_major_formatter(y_formatter)
    
    pyplot.xlabel(r'$x$')
    pyplot.ylabel(r'$y$')
    pyplot.axis('square')
    pyplot.xlim(xmin-x_std, xmax+x_std)
    pyplot.ylim(ymin-y_std, ymax+y_std)
    #pyplot.legend(loc ="upper left", bbox_to_anchor=(0.5, 0.49))

    if learning_type_flag == 'Chosen':
        pyplot.savefig(saved_img_path_reg_gn_given+'/img_visualize_hyperplane_hybrid_model_'+situation+'_'+jeu+'_reg_gn_given.png', dpi=500, bbox_inches='tight', transparent=True)
    else:
        pyplot.savefig(saved_img_path_reg_gn_search+'/img_visualize_hyperplane_hybrid_model_'+situation+'_'+jeu+'_reg_gn_search.png', dpi=500, bbox_inches='tight', transparent=True)

    figsize = (1,6)
    pyplot.figure(figsize=figsize, dpi=200)
    ax_leg = pyplot.gca()
    ax_leg.legend(*ax.get_legend_handles_labels(), ncol=2, loc='center', fontsize=9, facecolor='white', edgecolor='black', framealpha=0.05)
    ax_leg.axis('off')
    if learning_type_flag == 'Chosen':
        pyplot.savefig(saved_img_path_reg_gn_given+'/img_MoTLE_legend_reg_gn_given.png', dpi=1000, bbox_inches='tight', transparent=True)
    else:
        pyplot.savefig(saved_img_path_reg_gn_search+'/img_MoTLE_legend_reg_gn_search.png', dpi=1000, bbox_inches='tight', transparent=True)
        